In [2]:
import pandas as pd
import numpy as np
from glob import glob
from transformers import BertTokenizer
import torch
from torch import nn
%matplotlib inline
import json
from tqdm import tqdm
from sklearn import metrics
import os

from model_1_to_1 import (
    BertChineseLinearPunc,
    BertChineseCNNreplaceBert,
    BertChineseBigStrideCNN,
    BertChineseSlimCNNBert,
    BertChineseSlimCNNBertLSTM,
    BertChineseEmbSlimCNNBert,
    BertChineseEmbSlimCNNlstmBert,
    BertChineseEmbSlimCNNlstmBertLSTM,
)
glob('models/*')
path = './models/20240624_143110/'

### Prepare

In [3]:
with open(os.path.join(path, 'hyperparameters.json'), 'r') as f:
    hyperparameters = json.load(f)

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese', do_lower_case=True)

punctuation_enc = {
    'O': 0,
    '，': 1,
    '。': 2,
    '？': 3
}

# punctuation_enc = {
#     'O': 0,
#     'PERIOD': 1
# }

# segment_size = hyperparameters['segment_size']
segment_size = 200
device="cpu"
output_size = len(punctuation_enc)
dropout = hyperparameters['dropout']
# bert_punc = BertChineseEmbSlimCNNlstmBertLSTM(segment_size, output_size, dropout, None).to(device)
bert_punc = BertChineseLinearPunc(segment_size, output_size, dropout, None).to(device)
bert_punc.load_state_dict(torch.load(os.path.join(path, 'model')))
bert_punc.eval();

#### Test

In [4]:
y_pred = []
y_true = []
inputs = "你最近有什么旅行计划吗听起来很棒你准备去哪些地方真羡慕你我也一直想去日本你已经订好机票和酒店了吗那你一定要去一次日式旅馆听说那里的温泉很棒"
tokens = tokenizer.tokenize(inputs)
x = tokenizer.convert_tokens_to_ids(tokens)
# print(x)
x=torch.tensor(x).unsqueeze(0)
output = bert_punc(x)
y_pred += list(output.argmax(dim=1).cpu().data.numpy().flatten())
# print(output.shape,x.shape)
# print(y_pred)

# Modify tokens based on y_pred
for i in range(len(y_pred)):
    if y_pred[i] != 0:
        tokens[i] += list(punctuation_enc.keys())[y_pred[i]]
        
# Untokenize the output
# untokenized_output = tokenizer.convert_tokens_to_string(tokens)
# print(untokenized_output)
untokenized_output = ''.join(tokens)
print(untokenized_output)


RuntimeError: The size of tensor a (4089) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
#test